<img style="border:2px solid black;" src="Results/accueil.png" width=100% border="5"> </center> 
# <span style="color:blue">Porous Media Flow modelling using Jupyter </span>
## <span style="color:red"> PYGRDECL based Upscaling notebook </span>

## <span style="color:blue"> 1. Create GRDECL ECLIPSE/PETREL Pillar Grid </span>

In [6]:
from GRDECL2VTK import *

# 1.1 Set physical dimensions in physDims (m)
physDims=[2000.0,1000,500]

# 1.2 Set model grid dimensions in gridDims
Nx=50;  Ny=12;  Nz=Nx
gridDims=[Nx,Ny,Nz]

# 1.3 Set corner point grid options
opt=buildCPGGrid_opt(\
disturbed=False,\
flat=True,\
fault_drop=200 , fault_nx=Nx//2)

# 1.4 Create empty GeologyModel - Build CPG
Model=GeologyModel()
Model.buildCPGGrid(physDims,gridDims,opt)

# 1.5 Set output name
OutputFilePath="Results/"
Basefilename="PILLAR_Grid"
FileExtension="GRDECL"
Model.fname=OutputFilePath+Basefilename + "." + FileExtension

     Creating Grid:
       Type: CornerPoint Grid
       Grid Dimensions (NX,NY,NZ): (50 x 12 x 50)
       Number Of Grid Cells: 30000
       Created: Fields: ***PERMX***PERMY***PERMZ***PORO***


In [7]:
# Compute Pressure for a specified Pressure in a given direction 
# Direction in : "ijk","i","j","k"
Press_inj=1 #m^3/s
direction="i"
Model.compute_TPFA_Pressure(Press_inj,direction,opt )

# # Convert to vtk and plot
plotter=Model.plot_scalar("Pressure",ITK=True)
plotter.show(True)

     New variable [Pressure] created with a value of 1.000000!
iz: 20
[Geometry] Converting GRDECL to Paraview Hexahedron mesh data....
     NumOfPoints 240000
     NumOfCells 30000
     .....Done!
[Output] Writing "Results/PILLAR_Grid.vtu" Paraview file....Done!


Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…